In [1]:
import numpy as np
def extEucl(m, n):
    """
    Возвращает тройку (d,u,v) таких, что d — наибольший общий делитель пары (m,n) и d=um+vn
    """
    (a, b) = (m, n)
    u1 = 1; v1 = 0
    u2 = 0; v2 = 1

    while b != 0:
        assert (a == u1*m + v1*n and b == u2*m + v2*n)

        q = a // b; r = a % b
        assert (a == q*b + r)

        (a, b) = (b, r)
        (u1, u2) = (u2, u1 - q*u2)
        (v1, v2) = (v2, v1 - q*v2)

    if a >= 0:
        return (a, u1, v1)
    else:
        return (-a, -u1, -v1)
    
def invmod(x, m):
    """
    Возвращает обратный к x элемент, если x обратим (т.е. целые числа x,m взаимно простые),
    иначе - исключение типа ValueError
    """
    assert(m != 0)
    (d, u, v) = extEucl(m, x)
    if d == 1:
        return v%m
    else:
        raise ValueError("Element is not invertible modulo m")

## Алиса формирует открытый и закрытый ключи
$r = (r_1, . . . , r_n)$ - супер последовательность

In [2]:
r = np.array((3, 11, 24, 50, 115))

Выбираем $A$ и $B$ такие, что $B>2r_n$ и $(A,B)=1$

In [3]:
A = 113
B = 250

Вычисляем $M_i = Ar_i \pmod{B}$ для $i\in[1,n]$

In [4]:
M=np.array(list(map(lambda x: (x*A)%B,r)))
assert np.array_equal(M, np.array([89, 243, 212, 150, 245])) 

#### Публичный ключ Алисы - $M$
## Боб формирует сообщение
Секретное сообщение $x$

In [5]:
x = np.array((1, 0, 1, 0, 1))
x

array([1, 0, 1, 0, 1])

Расчитываем $$S = x * M$$

In [6]:
S = (x * M).sum()
assert S==546

###### Отправляем $S$ по каналу связи
## Алиса расшифровывает послание

Вычисляем $A^{-1}$ по модулю $B$

In [7]:
A_=invmod(A,B)
assert A_== 177

Вычисляем $$S^′ ≡ A^{−1}S (mod B)$$

In [8]:
S_=(A_*S)%B
assert S_==142

Затем решаем $$S^′ = x * r$$ где $x$ - исходное сообщение

In [14]:
def find_x(r,S):
    x=np.zeros(r.shape)
    for i in range(len(r)-1,-1,-1):
        if S>=r[i]:
            x[i]=1
            S-=r[i]
        else:
            x[i]=0
    return x.astype("int")

In [15]:
x=find_x(r,S_)
print("Передаваемое сообщение:",x)

Передаваемое сообщение: [1 0 1 0 1]


## Функциональная реализация

In [26]:
def encrypt(x,M):
    return(x * M).sum()
def decrypt(S,A,B,r):
    A_=invmod(A,B)
    S_=(A_*S)%B
    return find_x(r,S_)
def hak():
    pass

In [27]:
r_t = np.array((3, 11, 24, 50, 115)) # секретный ключ Алисы
A_t = 113 # секретный ключ Алисы
B_t = 250 # секретный ключ Алисы
M_t=np.array(list(map(lambda x: (x*A_t)%B_t,r_t))) # открытый ключ Алисы

x_t = np.array((1, 0, 1, 0, 1)) # сообщение

In [28]:
S_t=encrypt(x_t,M_t)
d_t=decrypt(S_t,A_t,B_t,r_t)
hak_t=hak()
print("Исходное:",x_t,"\nЗашифрованное:",e_t,"\nРасшифрованное:",d_t,"\nВзоманное:",hak_t)

Исходное: [1 0 1 0 1] 
Зашифрованное: 546 
Расшифрованное: [1 0 1 0 1] 
Взоманное: None
